In [1]:
%load_ext autoreload

In [2]:
import sys
import os

sys.path.insert(0, '../../model_sequence_labeling/lib/src')
os.environ['CUDA_LAUNCH_BLOCKING'] = "0" # for debugging
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [3]:
from tqdm import tqdm
from torch.utils.data import DataLoader
from torch.nn import functional as F
from transformers import AutoConfig, AutoTokenizer, get_polynomial_decay_schedule_with_warmup
from itertools import chain

import torch
import random
import os, sys
import numpy as np
import argparse
import time
import json
import pandas as pd

from glob import glob
from sklearn.model_selection import train_test_split
from tabulate import tabulate

In [4]:
import torch 
import random
torch.manual_seed(123)
torch.cuda.manual_seed(123)
np.random.seed(123)
random.seed(123)

In [5]:
%autoreload
from multi_turn_bert import MultiTurnBert
from custom_dataset import CustomDataset, process_df
import main as seq_labeling

from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

In [6]:
def load_ds(ds_path):
    df = pd.read_pickle(ds_path)

    #Aligning the 5-levels labels to eli5 ones
        
    #'(D06) To answer - Other' -> '(D06) Answer - Other'
    #'(D07) To provide agreement statement' -> '(D07) Agreement'
    #'(D08) To provide disagreement statement' -> '(D08) Disagreement'
    #'(D10) Other' -> '(D09) Other'
    #'(D09) To provide informing statement' -> (D10) To provide informing statement
    
    
    # (E10) Other -> (E09) Other 
    # (E09) Introducing Extraneous Information -> (E10) Introducing Extraneous Information
    
    df['exp_act_label'] = df.exp_act_label.apply(lambda x: '(E10) Other' if x == '(E09) Other' else x)
    df['exp_act_label'] = df.exp_act_label.apply(lambda x: '(E09) Introducing Extraneous Information' if x == '(E10) Introducing Extraneous Information' else x)

    df['dlg_act_label'] = df.dlg_act_label.apply(lambda x: '(D09) Other' if x == '(D10) Other' else x)
    df['dlg_act_label'] = df.dlg_act_label.apply(lambda x: '(D10) To provide informing statement' if x == '(D09) To provide informing statement' else x)
    
    df['dlg_act_label'] = df.dlg_act_label.apply(lambda x: '(D06) Answer - Other' if x == '(D06) To answer - Other' else x)
    df['dlg_act_label'] = df.dlg_act_label.apply(lambda x: '(D07) Agreement' if x == '(D07) To provide agreement statement' else x)
    df['dlg_act_label'] = df.dlg_act_label.apply(lambda x: '(D08) Disagreement' if x == '(D08) To provide disagreement statement' else x)
    

    df['turn_text_with_topic'] = df.apply(lambda row: {
                                        'author': row['turn_text']['author'], 
                                        'text'  : row['topic'].replace('_', ' ') + ' [SEP] ' +  row['turn_text']['text']
                                       } ,axis=1)

    return df

### Train to predict explanation act:

Training is in src-py/trune_label_prediction_with_bert_seq

### Predict and Evaluate:

In [7]:
folds_dict = json.load(open('../data/topic_folds.json'))

In [8]:
fivelvls_annotation_df = load_ds('../../data/five_levels_ds/annotation-results/MACE-measure/final_mace_predictions.pkl')
eli5_annotation_df     = load_ds('../../data/eli5_ds/annotation-results/MACE-measure/final_mace_predictions_training.pkl')

In [9]:
fivelvls_annotation_df['ds'] = ['5lvls'] * len(fivelvls_annotation_df)
eli5_annotation_df['ds'] = ['eli5'] * len(eli5_annotation_df)
dlgs_df = pd.concat([fivelvls_annotation_df, eli5_annotation_df])

In [18]:
def evaluate_fold(fold_path, df, fold_idx, input_clm, output_clm, label_clm='exp_act_label', ds_names=[], model_name='bert-base-uncased'):
    label_dictionary = {int(l[2:4])-1 : l for l in  df[label_clm].unique()}
    
    print(label_dictionary)
    print(fold_path)
    fold_topics = []
    if 'eli5' in ds_names:
        fold_topics += folds_dict['test']['eli5'][fold_idx]
    if '5lvls' in ds_names:
        fold_topics += folds_dict['test']['5lvls'][fold_idx]
    
    test_df  = df[df.topic.isin(fold_topics)]
    
    #Prepare the dataframe
    grouped_df = test_df.groupby('task_id').agg({
                                'turn_text': lambda rows: list(rows),
                                 input_clm: lambda rows: list(rows),
                                'topic': lambda rows: list(rows)[0],
                                'topic_func_label': lambda rows: list(rows),
                                'dlg_act_label': lambda rows: list(rows),
                                'exp_act_label': lambda rows: list(rows)}).reset_index()

    grouped_df['label'] = grouped_df[label_clm].apply(lambda labels: [int(x[2:4])-1 for x in labels]) #making labels parasable as integers

    #Prepare the dataset
    print(model_name)
    tokenizer   = AutoTokenizer.from_pretrained(model_name)
    bert_config = AutoConfig.from_pretrained(model_name)

    num_new_tokens = tokenizer.add_special_tokens(
        {
            'additional_special_tokens': [args.sp1_token, args.sp2_token, args.start_token]
        }
    )

    vocab=tokenizer.get_vocab()

    args.vocab_size = len(vocab)
    args.cls_token = tokenizer.cls_token
    args.sep_token = tokenizer.sep_token
    args.pad_token = tokenizer.pad_token
    args.cls_id = vocab[args.cls_token]
    args.sep_id = vocab[args.sep_token]
    args.pad_id = vocab[args.pad_token]
    args.sp1_id = vocab[args.sp1_token]
    args.sp2_id = vocab[args.sp2_token]
    args.start_id = vocab[args.start_token]
    args.o_id   = -1
    args.hidden_size = bert_config.hidden_size
    args.p_dim = args.hidden_size
    args.max_len = min(args.max_len, bert_config.max_position_embeddings)
    args.num_classes=len(label_dictionary)
    
    test_ds = CustomDataset(grouped_df, tokenizer, input_clm, args, tokenizer.get_vocab())
    eval_loader = DataLoader(test_ds, batch_size=args.batch_size, num_workers=args.num_workers, pin_memory=True)


    #Load the model
    model = MultiTurnBert(args).to(args.device)
    model.load_state_dict(torch.load(fold_path + 'best_model'))
    model.eval()

    eval_losses, eval_ys, eval_outputs, eval_lens = [], [], [], []
    with torch.no_grad():
        for i, batch in enumerate(tqdm(eval_loader)):
            batch_x, batch_turns, batch_labels, true_turn_lens = batch

            batch_x, batch_turns, batch_labels = \
                batch_x.to(args.device), batch_turns.to(args.device), batch_labels.to(args.device)

            _, outputs = model(batch_x, args.pad_id, batch_turns, turns_labels=batch_labels)  # (), (B, L)

            eval_ys.append(batch_labels.detach())
            eval_outputs.append(outputs)
            eval_lens.append(true_turn_lens)

        eval_preds, eval_trues = [], []
        for i in range(len(eval_ys)):
            pred_batch, true_batch, batch_lens = eval_outputs[i], eval_ys[i], eval_lens[i]

            batch_lens = batch_lens.tolist()  # (B)


            pred_batch = [batch[:batch_lens[b]] for b, batch in enumerate(pred_batch)]
            true_batch = [batch[:batch_lens[b]] for b, batch in enumerate(true_batch.tolist())]

            assert len(pred_batch) == len(true_batch)
            eval_preds += pred_batch
            eval_trues += true_batch


        #flatten preds and labels
        eval_preds = [item[-1] for item in eval_preds]
        eval_trues = [item[-1] for item in eval_trues]

        #print(eval_preds)
        #print(eval_trues)
        score = f1_score(eval_trues, eval_preds, average ='macro')
        test_df[output_clm] = [label_dictionary[x] for x in eval_preds]

    return score, test_df

def evaluate_model(models_path, label_clm, label_model, model_type='bert-base-uncased'):
    f1_scores = {}

    for model_name in ['eli5_models', '5lvls_models', 'all_models']:
        model_scores = []
        for ds_names in [['eli5'], ['5lvls'], ['5lvls', 'eli5']]:
            temp_scores = []
            for fold in range(5):
                s, _ = evaluate_fold('{}/{}/{}/model/fold-{}/'.format(models_path, label_model, model_name, fold), dlgs_df.copy(), fold, 'turn_text_with_topic', label_clm + "_predictions", label_clm, ds_names=ds_names, model_name=model_type)
                temp_scores.append(s)
            
            print(temp_scores)
            s = np.mean(temp_scores)
            model_scores.append(round(s, 2))

        f1_scores[model_name] = model_scores
    
    return f1_scores

In [17]:
parser = argparse.ArgumentParser()
args = argparse.Namespace(turn_type='multi', pooling='cls', sp1_token='[EXPLAINER]', sp2_token='[EXPLAINEE]', 
                          bert_type='bert-base-uncased',
                          max_len=256, max_turns=56, dropout=0.1, device='cuda', learning_rate=2e-5, warmup_ratio=0.01,
                          batch_size=2, num_workers=2, num_epochs=5, num_classes=10, ckpt_dir='topic-func-cross-val-models', planning=False, start_token='[START]')

In [15]:
len({9: '(D10) To provide informing statement', 6: '(D07) Agreement', 0: '(D01) To ask a check question', 4: '(D05) To answer a question by disconfirming', 1: '(D02) To ask what/how question', 5: '(D06) Answer - Other', 3: '(D04) To answer a question by confirming', 8: '(D09) Other', 2: '(D03) To ask other kind of questions', 7: '(D08) Disagreement'})

10

In [14]:
dlg_act_f1_scores = evaluate_model('../data/turn-label-models/seq/bert-base-uncased/', 'dlg_act_label', 'dlg_act_label_prediction')

{9: '(D10) To provide informing statement', 6: '(D07) Agreement', 0: '(D01) To ask a check question', 4: '(D05) To answer a question by disconfirming', 1: '(D02) To ask what/how question', 5: '(D06) Answer - Other', 3: '(D04) To answer a question by confirming', 8: '(D09) Other', 2: '(D03) To ask other kind of questions', 7: '(D08) Disagreement'}
../data/turn-label-models/seq/bert-base-uncased//dlg_act_label_prediction/eli5_models/model/fold-0/
bert-base-uncased


0it [00:00, ?it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (725 > 512). Running this sequence through the model will result in indexing errors
152it [00:00, 429.94it/s]
100%|██████████| 152/152 [00:00<00:00, 5155.46it/s]


1492 samples are prepared


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RuntimeError: Error(s) in loading state_dict for MultiTurnBert:
	size mismatch for position_wise_ff.weight: copying a param with shape torch.Size([11, 768]) from checkpoint, the shape in current model is torch.Size([10, 768]).
	size mismatch for position_wise_ff.bias: copying a param with shape torch.Size([11]) from checkpoint, the shape in current model is torch.Size([10]).
	size mismatch for crf.start_transitions: copying a param with shape torch.Size([11]) from checkpoint, the shape in current model is torch.Size([10]).
	size mismatch for crf.end_transitions: copying a param with shape torch.Size([11]) from checkpoint, the shape in current model is torch.Size([10]).
	size mismatch for crf.transitions: copying a param with shape torch.Size([11, 11]) from checkpoint, the shape in current model is torch.Size([10, 10]).

In [26]:
exp_act_f1_scores = evaluate_model('../data/turn-label-models/seq/bert-base-uncased/', 'exp_act_label', 'exp_act_label_prediction')

{1: '(E02) Testing prior knowledge', 2: '(E03) Provide an explanation', 4: '(E05) Signaling understanding', 8: '(E09) Introducing Extraneous Information', 6: '(E07) Providing Feedback', 0: '(E01) Testing understanding', 9: '(E10) Other', 3: '(E04) Ask for an explanation', 7: '(E08) Providing Assessment', 5: '(E06) Signaling non-understanding'}
../data/turn-label-models/seq/bert-base-uncased//exp_act_label_prediction/eli5_ds_models/model/fold-0/
bert-base-uncased


0it [00:00, ?it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (725 > 512). Running this sequence through the model will result in indexing errors
152it [00:00, 328.05it/s]
100%|██████████| 152/152 [00:00<00:00, 4059.49it/s]


1492 samples are prepared


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


FileNotFoundError: [Errno 2] No such file or directory: '../data/turn-label-models/seq/bert-base-uncased//exp_act_label_prediction/eli5_ds_models/model/fold-0/best_model'

In [27]:
topic_func_f1_scores = evaluate_model('../data/turn-label-models/seq/bert-base-uncased/', 'topic_func_label', 'topic_func_label_prediction')

{3: '(T04) Other - No topic was introduced', 0: '(T01) It is the main topic', 2: '(T03) A related topic', 1: '(T02) A subtopic'}
../data/turn-label-models/seq/bert-base-uncased//topic_func_label_prediction/eli5_ds_models/model/fold-0/
bert-base-uncased


0it [00:00, ?it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (725 > 512). Running this sequence through the model will result in indexing errors
152it [00:00, 234.61it/s]
100%|██████████| 152/152 [00:00<00:00, 4009.09it/s]


1492 samples are prepared


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


FileNotFoundError: [Errno 2] No such file or directory: '../data/turn-label-models/seq/bert-base-uncased//topic_func_label_prediction/eli5_ds_models/model/fold-0/best_model'

In [16]:
print(tabulate([[x[0][0]] + x[0][1] + x[1][1] + x[2][1] for x in zip(exp_act_f1_scores.items(), dlg_act_f1_scores.items(), topic_func_f1_scores.items())], 
               headers=['Approach', 'ELI5', '5lvls', 'ALL', 'ELI5', '5lvls', 'ALL', 'ELI5', '5lvls', 'ALL']))

Approach           ELI5    5lvls    ALL    ELI5    5lvls    ALL    ELI5    5lvls    ALL
---------------  ------  -------  -----  ------  -------  -----  ------  -------  -----
eli5_ds_models    0.4      0.198  0.223   0.361    0.233  0.351   0.308    0.324  0.342
5lvls_ds_models   0.18     0.36   0.291   0.077    0.447  0.252   0.321    0.489  0.437
mixed_ds_models   0.313    0.407  0.393   0.388    0.449  0.528   0.342    0.521  0.476


#### Predicting on RoBERTa:

In [15]:
args = argparse.Namespace(turn_type='multi', pooling='cls', sp1_token='[EXPLAINER]', sp2_token='[EXPLAINEE]', 
                          bert_type='roberta-base',
                          max_len=256, max_turns=56, dropout=0.1, device='cuda', learning_rate=2e-5, warmup_ratio=0.01,
                          batch_size=2, num_workers=2, num_epochs=5, num_classes=10, ckpt_dir='topic-func-cross-val-models', planning=False, start_token='[START]')

In [16]:
roberta_dlg_act_f1_scores = evaluate_model('../data/roberta_seq', 'dlg_act_label', 'dlg_act_prediction', model_type='roberta-base')

{8: '(D09) Other', 6: '(D07) Agreement', 0: '(D01) To ask a check question', 4: '(D05) To answer a question by disconfirming', 1: '(D02) To ask what/how question', 5: '(D06) Answer - Other', 3: '(D04) To answer a question by confirming', 2: '(D03) To ask other kind of questions', 7: '(D08) Disagreement', 9: '(D10) To provide informing statement'}
../data/roberta_seq/dlg_act_prediction/eli5_ds_models/model/fold-0/
roberta-base


0it [00:00, ?it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (528 > 512). Running this sequence through the model will result in indexing errors
152it [00:00, 459.56it/s]
100%|██████████| 152/152 [00:00<00:00, 3821.62it/s]


1492 samples are prepared


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 746/746 [01:33<00:00,  7.95it/s]
<ipython-input-12-27a8ff436bcb>:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.or

{8: '(D09) Other', 6: '(D07) Agreement', 0: '(D01) To ask a check question', 4: '(D05) To answer a question by disconfirming', 1: '(D02) To ask what/how question', 5: '(D06) Answer - Other', 3: '(D04) To answer a question by confirming', 2: '(D03) To ask other kind of questions', 7: '(D08) Disagreement', 9: '(D10) To provide informing statement'}
../data/roberta_seq/dlg_act_prediction/eli5_ds_models/model/fold-1/
roberta-base


...

0it [00:00, ?it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (558 > 512). Running this sequence through the model will result in indexing errors
43it [00:00, 587.81it/s]
100%|██████████| 43/43 [00:00<00:00, 5521.86it/s]


359 samples are prepared


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 180/180 [00:24<00:00,  7.39it/s]

[0.4612762312262923, 0.5635712204328517, 0.5870294143890264, 0.5472888069272969, 0.4988352465054506]



<ipython-input-12-27a8ff436bcb>:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[output_clm] = [label_dictionary[x] for x in eval_preds]


In [17]:
roberta_exp_act_f1_scores = evaluate_model('../data/roberta_seq', 'exp_act_label', 'exp_act_prediction', model_type='roberta-base')

{1: '(E02) Testing prior knowledge', 2: '(E03) Provide an explanation', 4: '(E05) Signaling understanding', 9: '(E10) Introducing Extraneous Information', 6: '(E07) Providing Feedback', 0: '(E01) Testing understanding', 8: '(E09) Other', 3: '(E04) Ask for an explanation', 7: '(E08) Providing Assessment', 5: '(E06) Signaling non-understanding'}
../data/roberta_seq/exp_act_prediction/eli5_ds_models/model/fold-0/
roberta-base


0it [00:00, ?it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (528 > 512). Running this sequence through the model will result in indexing errors
152it [00:00, 446.64it/s]
100%|██████████| 152/152 [00:00<00:00, 4598.06it/s]


1492 samples are prepared


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 746/746 [01:33<00:00,  7.94it/s]
<ipython-input-12-27a8ff436bcb>:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.or

{1: '(E02) Testing prior knowledge', 2: '(E03) Provide an explanation', 4: '(E05) Signaling understanding', 9: '(E10) Introducing Extraneous Information', 6: '(E07) Providing Feedback', 0: '(E01) Testing understanding', 8: '(E09) Other', 3: '(E04) Ask for an explanation', 7: '(E08) Providing Assessment', 5: '(E06) Signaling non-understanding'}
../data/roberta_seq/exp_act_prediction/eli5_ds_models/model/fold-1/
roberta-base


...

{1: '(E02) Testing prior knowledge', 2: '(E03) Provide an explanation', 4: '(E05) Signaling understanding', 9: '(E10) Introducing Extraneous Information', 6: '(E07) Providing Feedback', 0: '(E01) Testing understanding', 8: '(E09) Other', 3: '(E04) Ask for an explanation', 7: '(E08) Providing Assessment', 5: '(E06) Signaling non-understanding'}
../data/roberta_seq/exp_act_prediction/mixed_ds_models/model/fold-4/
roberta-base


0it [00:00, ?it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (558 > 512). Running this sequence through the model will result in indexing errors
43it [00:00, 593.29it/s]
100%|██████████| 43/43 [00:00<00:00, 5797.15it/s]


359 samples are prepared


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 180/180 [00:27<00:00,  6.54it/s]
<ipython-input-12-27a8ff436bcb>:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.or

[0.3664889764968619, 0.40928403874468133, 0.4144258479832967, 0.35695628619216707, 0.35381534359099137]


In [18]:
roberta_topic_func_f1_scores = evaluate_model('../data/roberta_seq', 'topic_func_label', 'topic_func_prediction', model_type='roberta-base')

{3: '(T04) Other - No topic was introduced', 0: '(T01) It is the main topic', 2: '(T03) A related topic', 1: '(T02) A subtopic'}
../data/roberta_seq/topic_func_prediction/eli5_ds_models/model/fold-0/
roberta-base


0it [00:00, ?it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (528 > 512). Running this sequence through the model will result in indexing errors
152it [00:00, 455.35it/s]
100%|██████████| 152/152 [00:00<00:00, 4082.57it/s]


1492 samples are prepared


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 746/746 [01:36<00:00,  7.69it/s]
<ipython-input-12-27a8ff436bcb>:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.or

{3: '(T04) Other - No topic was introduced', 0: '(T01) It is the main topic', 2: '(T03) A related topic', 1: '(T02) A subtopic'}
../data/roberta_seq/topic_func_prediction/eli5_ds_models/model/fold-1/
roberta-base


...

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 181/181 [00:27<00:00,  6.62it/s]
<ipython-input-12-27a8ff436bcb>:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.or

{3: '(T04) Other - No topic was introduced', 0: '(T01) It is the main topic', 2: '(T03) A related topic', 1: '(T02) A subtopic'}
../data/roberta_seq/topic_func_prediction/5lvls_ds_models/model/fold-2/
roberta-base


15it [00:00, 230.55it/s]
100%|██████████| 15/15 [00:00<00:00, 1983.81it/s]


421 samples are prepared


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
 70%|██████▉   | 147/211 [00:23<00:07,  8.15it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg

In [19]:
print(tabulate([[x[0][0]] + x[0][1] + x[1][1] + x[2][1] for x in zip(roberta_exp_act_f1_scores.items(), roberta_dlg_act_f1_scores.items(), roberta_topic_func_f1_scores.items())], 
               headers=['Approach', 'ELI5', '5lvls', 'ALL', 'ELI5', '5lvls', 'ALL', 'ELI5', '5lvls', 'ALL']))

Approach           ELI5    5lvls    ALL    ELI5    5lvls    ALL    ELI5    5lvls    ALL
---------------  ------  -------  -----  ------  -------  -----  ------  -------  -----
eli5_ds_models    0.378    0.203  0.242   0.357    0.188  0.327   0.336    0.376  0.369
5lvls_ds_models   0.168    0.345  0.269   0.093    0.473  0.256   0.247    0.549  0.408
mixed_ds_models   0.319    0.375  0.38    0.39     0.467  0.532   0.371    0.553  0.494


-----------

### Predicting labels for the final quality assessment:

- Best Performing Models:
    - Explanation Moves: ELI-5 trained BERT-Seq
    - Dialogue Acts: Both trained RoBERTa
    - Topic Func: ELI-5 trained RoBERTa

In [68]:
%autoreload

from sklearn.model_selection import KFold
from multi_turn_bert import MultiTurnBert
from custom_dataset import CustomDataset, process_df
import main as seq_labeling
import pickle

from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

In [75]:
eli5_annotation_df     = pd.read_pickle('../data/final_mace_predictions_testing_with_predictions.pkl') # This file was prepared in predicting-turn-labels-with-bert.ipynb

In [82]:
parser = argparse.ArgumentParser()
args = argparse.Namespace(turn_type='multi', pooling='cls', sp1_token='[EXPLAINER]', sp2_token='[EXPLAINEE]', bert_type='bert-base-uncased',
                          max_len=256, max_turns=56, dropout=0.1, device='cuda', learning_rate=2e-5, warmup_ratio=0.01,
                          batch_size=2, num_workers=2, num_epochs=5, num_classes=-1, ckpt_dir='../data/seq-labeling/model', planning=False, start_token='[START]')

In [78]:
def model_predictions(model, eval_loader):
    eval_losses, eval_ys, eval_outputs, eval_lens = [], [], [], []
    with torch.no_grad():
        for i, batch in enumerate(tqdm(eval_loader)):
            batch_x, batch_turns, batch_labels, true_turn_lens = batch

            batch_x, batch_turns, batch_labels = \
                batch_x.to(args.device), batch_turns.to(args.device), batch_labels.to(args.device)

            _, outputs = model(batch_x, args.pad_id, batch_turns, turns_labels=batch_labels)  # (), (B, L)

            eval_ys.append(batch_labels.detach())
            eval_outputs.append(outputs)
            eval_lens.append(true_turn_lens)


        eval_preds, eval_trues = [], []
        for i in range(len(eval_ys)):
            pred_batch, true_batch, batch_lens = eval_outputs[i], eval_ys[i], eval_lens[i]

            batch_lens = batch_lens.tolist()  # (B)


            pred_batch = [batch[:batch_lens[b]] for b, batch in enumerate(pred_batch)]
            true_batch = [batch[:batch_lens[b]] for b, batch in enumerate(true_batch.tolist())]

            assert len(pred_batch) == len(true_batch)
            eval_preds += pred_batch
            eval_trues += true_batch


        #flatten preds and labels
        eval_preds = [item[-1] for item in eval_preds]
        eval_trues = [item[-1] for item in eval_trues]

        #print(eval_preds)
        #print(eval_trues)
        score = f1_score(eval_trues, eval_preds, average ='macro')
        print('f1-score: {}'.format(score))
        return eval_preds
        
def ensemble_generate(model_path, test_df, input_clm, args, label_clm='exp_act_label'):
    
        
    label_dictionary = {int(l[2:4])-1 : l for l in  test_df[label_clm].unique()}
    print(label_dictionary)
        
    #Prepare the dataframe
    grouped_df = test_df.groupby('task_id').agg({
                                'turn_text': lambda rows: list(rows),
                                 input_clm: lambda rows: list(rows),
                                'topic': lambda rows: list(rows)[0],
                                'topic_func_label': lambda rows: list(rows),
                                'dlg_act_label': lambda rows: list(rows),
                                'exp_act_label': lambda rows: list(rows)}).reset_index()

    grouped_df['label'] = grouped_df[label_clm].apply(lambda labels: [int(x[2:4])-1 for x in labels]) #making labels parasable as integers
    
    #Prepare the dataset
    tokenizer   = AutoTokenizer.from_pretrained(args.bert_type)
    bert_config = AutoConfig.from_pretrained(args.bert_type)

    num_new_tokens = tokenizer.add_special_tokens(
        {
            'additional_special_tokens': [args.sp1_token, args.sp2_token, args.start_token]
        }
    )

    vocab=tokenizer.get_vocab()

    args.vocab_size = len(vocab)
    args.cls_token = tokenizer.cls_token
    args.num_classes= len(label_dictionary)
    args.sep_token = tokenizer.sep_token
    args.pad_token = tokenizer.pad_token
    args.cls_id = vocab[args.cls_token]
    args.sep_id = vocab[args.sep_token]
    args.pad_id = vocab[args.pad_token]
    args.sp1_id = vocab[args.sp1_token]
    args.sp2_id = vocab[args.sp2_token]
    args.start_id = vocab[args.start_token]
    args.o_id   = -1
    args.hidden_size = bert_config.hidden_size
    args.p_dim = args.hidden_size
    args.max_len = min(args.max_len, bert_config.max_position_embeddings)
    
    test_ds = CustomDataset(grouped_df, tokenizer, input_clm, args, tokenizer.get_vocab())
    eval_loader = DataLoader(test_ds, batch_size=args.batch_size, num_workers=args.num_workers, pin_memory=True)

    
    models = []
    for fold_path in glob(model_path):
        #Load the model
        print('Loading: {}'.format(fold_path))
        args.ckpt_dir = fold_path
        print(args)
        model = MultiTurnBert(args).to(args.device)
        model.load_state_dict(torch.load(fold_path + '/best_model'))
        model.eval()
        models.append(model)
    
    models_preds = []
    for model in models:
        eval_loader = DataLoader(test_ds, batch_size=args.batch_size, num_workers=args.num_workers, pin_memory=True)
        models_preds.append(model_predictions(model, eval_loader))

    agg_predictions = [max(set(lst), key=lst.count) for lst in list(zip(*models_preds))] # [[1,2,..], [1,2,..]] -> [(1,1), (2,2),..] -> [1, 2]
    
    test_df[label_clm + '_predictions'] = [label_dictionary[x] for x in agg_predictions]
    
    print('Prec. {} Rec. {}, F1-score {}'.format(
        precision_score(test_df['{}_predictions'.format(label_clm)], test_df[label_clm], average='macro'),
        recall_score(test_df['{}_predictions'.format(label_clm)], test_df[label_clm], average='macro'),
        f1_score(test_df['{}_predictions'.format(label_clm)], test_df[label_clm], average='macro')
    ))
    
    return test_df

In [79]:
test_df = ensemble_generate('../data/bert_seq/exp_act_prediction/eli5_ds_models/model/fold-*', eli5_annotation_df, 'turn_text_with_topic', args, label_clm='exp_act_label')

{3: '(E04) Ask for an explanation', 2: '(E03) Provide an explanation', 5: '(E06) Signaling non-understanding', 6: '(E07) Providing Feedback', 7: '(E08) Providing Assessment', 4: '(E05) Signaling understanding', 8: '(E09) Other', 9: '(E10) Introducing Extraneous Information', 0: '(E01) Testing understanding', 1: '(E02) Testing prior knowledge'}


0it [00:00, ?it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (862 > 512). Running this sequence through the model will result in indexing errors
122it [00:00, 405.05it/s]
100%|██████████| 122/122 [00:00<00:00, 4303.12it/s]


1074 samples are prepared
Loading: ../data/bert_seq/exp_act_prediction/eli5_ds_models/model/fold-4
Namespace(batch_size=4, bert_type='bert-base-uncased', ckpt_dir='../data/bert_seq/exp_act_prediction/eli5_ds_models/model/fold-4', cls_id=101, cls_token='[CLS]', device='cuda', dropout=0.1, hidden_size=768, learning_rate=2e-05, max_len=256, max_turns=56, num_classes=10, num_epochs=5, num_workers=2, o_id=-1, p_dim=768, pad_id=0, pad_token='[PAD]', planning=False, pooling='cls', sep_id=102, sep_token='[SEP]', sp1_id=30522, sp1_token='[EXPLAINER]', sp2_id=30523, sp2_token='[EXPLAINEE]', start_id=30524, start_token='[START]', turn_type='multi', vocab_size=30525, warmup_ratio=0.01)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Loading: ../data/bert_seq/exp_act_prediction/eli5_ds_models/model/fold-3
Namespace(batch_size=4, bert_type='bert-base-uncased', ckpt_dir='../data/bert_seq/exp_act_prediction/eli5_ds_models/model/fold-3', cls_id=101, cls_token='[CLS]', device='cuda', dropout=0.1, hidden_size=768, learning_rate=2e-05, max_len=256, max_turns=56, num_classes=10, num_epochs=5, num_workers=2, o_id=-1, p_dim=768, pad_id=0, pad_token='[PAD]', planning=False, pooling='cls', sep_id=102, sep_token='[SEP]', sp1_id=30522, sp1_token='[EXPLAINER]', sp2_id=30523, sp2_token='[EXPLAINEE]', start_id=30524, start_token='[START]', turn_type='multi', vocab_size=30525, warmup_ratio=0.01)


...

100%|██████████| 269/269 [01:00<00:00,  4.44it/s]


f1-score: 0.22831691122341563


100%|██████████| 269/269 [01:00<00:00,  4.44it/s]

f1-score: 0.2190444661844179
Prec. 0.24303831967278403 Rec. 0.22745843152311224, F1-score 0.23448281679263872



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [81]:
test_df.to_pickle('../data/final_mace_predictions_testing_with_predictions.pkl')